In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
tqdm.pandas()

/root/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('gen_prev_ans_ques_set...')

gen_prev_ans_ques_set...


In [3]:
data = pd.read_pickle(os.path.join(DATA_PATH, 'invite_data.pkl'))
prev_ans_ques = pd.read_pickle(os.path.join(DATA_PATH, 'prev_ans_ques.pkl'))
question_info = pd.read_pickle(os.path.join(DATA_PATH, 'question_info.pkl'))

In [4]:
data = pd.concat([data, prev_ans_ques], axis=1)

In [5]:
data['prev_ans_ques'] = data['prev_ans_ques'].progress_apply(lambda x: x if type(x) == list else [])

100%|██████████| 10630845/10630845 [00:14<00:00, 731919.24it/s] 


In [6]:
tmp = question_info[['qid', 'title_w', 'desc_w']]
data = data.merge(tmp, 'left', 'qid')
data.head()

,iday,ihour,itime,label,qid,uid,prev_ans_ques,title_w,desc_w
0,3865,22,92782,0.0,2166419046,401693808,[],"[11058, 272, 2202, 431, 951, 243, 3828, 3037, ...","[3828, 14870, 3037, 2450, 692, 1688, 53547, 24..."
1,3844,11,92267,0.0,1550017551,3392373099,[],"[149, 79, 5210, 22869]","[429, 2709, 1465, 6965]"
2,3862,15,92703,0.0,604029601,2317670257,"[3034061280, 1309562090]","[31489, 9218, 3440, 243]","[3046, 221, 3440, 730, 31489, 9218, 1278, 415]"
3,3849,11,92387,0.0,2350061229,1618461867,[],"[973, 64958, 8583, 2269, 628, 565, 2200, 590, ...",[0]
4,3867,4,92812,0.0,2443223942,3544409350,[],"[554, 28208, 396, 51]",[0]


In [7]:
tmp.set_index('qid', inplace=True)
tmp.head()

,title_w,desc_w
qid,,
2234111670,"[22414, 963, 10458]",[0]
760329790,"[12677, 16829, 15201, 6419, 101839]","[1296, 2118, 12677, 16829, 15201, 6419, 101839..."
741313548,"[700, 2781, 3280, 81215]","[732, 24400, 48321, 39608, 20788, 219486, 1183..."
3481466230,"[3312, 1823, 1505, 638, 166, 461]","[6642, 4214, 3312, 1505, 2205, 232, 294, 7177,..."
3966197028,"[700, 895, 2253]",[0]


In [8]:
prev_title_w = []
prev_desc_w = []
for prev_ques in tqdm(data['prev_ans_ques'].values):
    t = []
    d = []
    for q in prev_ques:
        t.append(tmp.loc[q, 'title_w'])
        d.append(tmp.loc[q, 'desc_w'])
    prev_title_w.append(t)
    prev_desc_w.append(d)

100%|██████████| 10630845/10630845 [24:07<00:00, 7342.15it/s] 


In [9]:
data['prev_title_w'] = prev_title_w
data['prev_desc_w'] = prev_desc_w

In [10]:
data.head()

,iday,ihour,itime,label,qid,uid,prev_ans_ques,title_w,desc_w,prev_title_w,prev_desc_w
0,3865,22,92782,0.0,2166419046,401693808,[],"[11058, 272, 2202, 431, 951, 243, 3828, 3037, ...","[3828, 14870, 3037, 2450, 692, 1688, 53547, 24...",[],[]
1,3844,11,92267,0.0,1550017551,3392373099,[],"[149, 79, 5210, 22869]","[429, 2709, 1465, 6965]",[],[]
2,3862,15,92703,0.0,604029601,2317670257,"[3034061280, 1309562090]","[31489, 9218, 3440, 243]","[3046, 221, 3440, 730, 31489, 9218, 1278, 415]","[[272, 263, 4534, 7742], [1116, 6761, 17922]]","[[272, 4534, 7742, 3738, 1277, 111, 2425, 4183..."
3,3849,11,92387,0.0,2350061229,1618461867,[],"[973, 64958, 8583, 2269, 628, 565, 2200, 590, ...",[0],[],[]
4,3867,4,92812,0.0,2443223942,3544409350,[],"[554, 28208, 396, 51]",[0],[],[]


In [11]:
def func(row):
    tw, dw, ptw, pdw = row['title_w'], row['desc_w'], row['prev_title_w'], row['prev_desc_w']
    tw, dw = set(tw), set(dw)
    com_ptw, com_pdw = [], []
    for t in ptw:
        t = set(t)
        com_ptw.append(len(tw & t))
    for d in pdw:
        d = set(d)
        com_pdw.append(len(dw & d))
    com_ptw = [0] if len(com_ptw) == 0 else com_ptw
    com_pdw = [0] if len(com_pdw) == 0 else com_pdw
    return np.array([np.max(com_ptw), np.min(com_ptw), np.mean(com_ptw), np.std(com_ptw),
            np.max(com_pdw), np.min(com_pdw), np.mean(com_pdw), np.std(com_pdw)])
        

def process(df):
    return np.vstack(df.progress_apply(lambda row: func(row),axis=1).values.tolist())

In [ ]:
with mp.Pool(8) as pool:
    ret = pool.map(process, np.array_split(data, 8))

In [ ]:
arr = np.vstack(ret)
print(arr.shape)

In [ ]:
user_ques_title_desc_com_stat = pd.DataFrame(arr, columns=[(tp + st) for tp in ['title_com_', 'desc_com_'] for st in ['max', 'min', 'mean', 'std']])
user_ques_title_desc_com_stat.head()

In [ ]:
user_ques_title_desc_com_stat['title_com_max/len'] = user_ques_title_desc_com_stat['title_com_max'] / data['title_w'].apply(len).values
user_ques_title_desc_com_stat['desc_com_max/len'] = user_ques_title_desc_com_stat['desc_com_max'] / data['desc_w'].apply(len).values
user_ques_title_desc_com_stat.head()

In [ ]:
user_ques_title_desc_com_stat.to_pickle(os.path.join(SAVE_PATH, 'user_ques_title_desc_com_stat.pkl'))

In [ ]:
word = pd.read_pickle(os.path.join('../pkl', 'word.pkl'))
word_embeddings = np.vstack(list(word['vector'].values))

def cos_sim(a, b):
    return 1 - spatial.distance.cosine(a, b)

def get_word_sim(i, j):
    if i == 0 or j == 0:
        return 0
    if i == j:
        return 1
    return cos_sim(word_embeddings[i-1], word_embeddings[j-1])

In [12]:
def process(df):
    sims_st = np.zeros((len(df), 8))
    for i, (tw, dw, ptw, pdw) in enumerate(tqdm(df[['title_w', 'desc_w', 'prev_title_w', 'prev_desc_w']].values)):
        tss = []
        dss = []
        
        for t2s in ptw:
            for t2 in t2s:
                for t1 in tw:
                    tss.append(get_word_sim(t1, t2))
        
        for d2s in pdw[-3:]:
            for d2 in d2s[:10]:
                for d1 in dw:
                    dss.append(get_word_sim(d1, d2))

        tss = ([0] if len(tss) == 0 else tss)
        dss = ([0] if len(dss) == 0 else dss)
        sims_st[i] = np.array([np.max(tss), np.min(tss), np.mean(tss), np.std(tss),
                               np.max(dss), np.min(dss), np.mean(dss), np.std(dss)])
    return sims_st

In [13]:
with mp.Pool(15) as pool:
    ret = pool.map(process, np.array_split(data, 30))
ret = np.vstack(ret)

  0%|          | 0/354361 [00:00<?, ?it/s]


NameError: name 'get_word_sim' is not defined

In [ ]:
cols = ['%s_sims_%s' % (t, st) for t in ['title', 'desc'] for st in ['max', 'min', 'mean', 'std']]
user_ques_title_desc_sims_stat = pd.DataFrame(ret, columns=cols)
print(user_ques_title_desc_sims_stat.shape)

In [ ]:
user_ques_title_desc_sims_stat.head()

In [ ]:
user_ques_title_desc_com_stat.to_pickle(os.path.join(SAVE_PATH, 'user_ques_title_desc_sims_stat.pkl'))

In [23]:
print(a)

In [24]:
gc.collect()

In [ ]:
toc = time.time()
print('Used time: %d' % int(toc-tic))